In [195]:
import os
import pandas
import numpy
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Load Files

In [196]:
train = pandas.read_csv("../data/train.csv",sep=",")
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Exploration

In [197]:
#Get the type of each columns
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [198]:
# Number of unique values
train.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [199]:
# Ratio of empty columns
train.isnull().sum()/len(train)

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

## label encoder 

In [200]:
def transform_categorical_values(column):
    le = preprocessing.LabelEncoder()
    return le.fit_transform(column)


## Variable Dummification

In [201]:
def get_dummies(dataset):
    df = pandas.DataFrame(index=dataset.index)
    for i in dataset.columns:
        df = pandas.concat([df, pandas.get_dummies(dataset[i])],axis = 1)
    return df

In [202]:
# get categorical values
train.columns[train.dtypes == numpy.object]

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [203]:
get_dummies(train[['Embarked','Sex']]).head()

,C,Q,S,female,male
0,0,0,1,0,1
1,1,0,0,1,0
2,0,0,1,1,0
3,0,0,1,1,0
4,0,0,1,0,1


# Feature engineering

## Embarked 

In [204]:
train.Embarked  = train.Embarked.fillna("S")


## Age 

In [205]:
train.Age = train.Age.copy()
age = numpy.mean(train.Age[(numpy.isnan(train.Age)==False)])
meanAge = round(numpy.mean(train.Age[(numpy.isnan(train.Age)==False)]),0)
print(meanAge)
train.Age = train.Age.fillna(meanAge)
train.Age = train.Age.astype(int)

30.0


## Transform age into categorical values

train.Age.describe()
train.Age = pandas.qcut(train.Age, [0, .25, .5, .75, 1],labels=[0,1,2,3])
train.Age = pandas.to_numeric(train.Age)
train.Age.head()

## Cabin 

In [206]:
tr = train.Cabin.fillna(value=0)
print("Number of non-zero = ",numpy.count_nonzero(tr))

Number of non-zero =  204


## SiibSpouse

In [207]:
train.SibSp.value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

## Parch 

In [208]:
train.Parch.describe()
train.Parch.value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

## Fare 

In [209]:
print(train.Fare.describe())
train.Fare = train.Fare.astype('int')
#train.Fare = pandas.qcut(train.Fare, [0, .25, .5, .75, 1],labels=[0,1,2,3])

train.Fare.head()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64


0     7
1    71
2     7
3    53
4     8
Name: Fare, dtype: int64

## Name Occurence

In [210]:
names = train.Name.str.split(",").str.get(0)
families = names.value_counts()
nameList = families.index.to_series()
nameOccurences = pandas.DataFrame(nameList)
nameOccurences['numbers'] = families
nameOccurences.columns = ['Name','Occs']
train['JustNames'] = names
print(train.head())
print(nameOccurences.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex  Age  SibSp  \
0                            Braund, Mr. Owen Harris    male   22      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38      1   
2                             Heikkinen, Miss. Laina  female   26      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35      1   
4                           Allen, Mr. William Henry    male   35      0   

   Parch            Ticket  Fare Cabin Embarked  JustNames  
0      0         A/5 21171     7   NaN        S     Braund  
1      0          PC 17599    71   C85        C    Cumings  
2      0  STON/O2. 3101282     7   NaN        S  Heikkinen  
3      0            113803    53  C123        S   Futrelle  
4      0            3

In [211]:
nameOccurences['Survived'] = 0
for row in nameOccurences.Name:
    nameOccurences.ix[nameOccurences.Name==row,'Survived'] = sum(train.Survived[train.JustNames == row])
nameOccurences['SurvivingRation'] = nameOccurences.Survived/nameOccurences.Occs
print(nameOccurences.head(10))

/home/keberger/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


                  Name  Occs  Survived  SurvivingRation
Andersson    Andersson     9         2         0.222222
Sage              Sage     7         0         0.000000
Carter          Carter     6         4         0.666667
Johnson        Johnson     6         3         0.500000
Goodwin        Goodwin     6         0         0.000000
Panula          Panula     6         0         0.000000
Skoog            Skoog     6         0         0.000000
Rice              Rice     5         0         0.000000
Smith            Smith     4         1         0.250000
Gustafsson  Gustafsson     4         0         0.000000


# Build dataset 

In [212]:
df = pandas.DataFrame(index = train.index)
df = pandas.concat([train[train.columns[train.dtypes==numpy.int]], get_dummies(train[['Embarked','Sex']])],axis = 1)
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,C,Q,S,female,male
0,1,0,3,22,1,0,7,0,0,1,0,1
1,2,1,1,38,1,0,71,1,0,0,1,0
2,3,1,3,26,0,0,7,0,0,1,1,0
3,4,1,1,35,1,0,53,0,0,1,1,0
4,5,0,3,35,0,0,8,0,0,1,0,1


In [213]:
train[train.columns[train.dtypes==numpy.int64]].head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22,1,0,7
1,2,1,1,38,1,0,71
2,3,1,3,26,0,0,7
3,4,1,1,35,1,0,53
4,5,0,3,35,0,0,8


## Remove passengerID 

In [214]:
df.drop('PassengerId',axis = 1,inplace = True)

## Display final dataset 

In [215]:
print(df.shape)
df.head()

(891, 11)


,Survived,Pclass,Age,SibSp,Parch,Fare,C,Q,S,female,male
0,0,3,22,1,0,7,0,0,1,0,1
1,1,1,38,1,0,71,1,0,0,1,0
2,1,3,26,0,0,7,0,0,1,1,0
3,1,1,35,1,0,53,0,0,1,1,0
4,0,3,35,0,0,8,0,0,1,0,1


# Save Data 

In [216]:
df.to_excel("../data/clean_train_dataset.xlsx", index = False, index_label= False)